In [ ]:
!pip install moviepy
!pip install assemblyai
!pip install librosa
!pip install pydub
!pip install noisereduce
!pip install soundfile
!pip install huggingface_hub
!pip install transformer
!pip install evaluate
!pip install jiwer
!pip install sentencepiece
!pip install tensorflow_io

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.3/157.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 

In [ ]:
# pip install moviepy

In [ ]:
from moviepy.editor import *
import librosa
import numpy as np
import matplotlib.pyplot as plt
import os
import assemblyai as aai
import pandas as pd
from pydub import AudioSegment
from pydub.silence import split_on_silence
import noisereduce as nr
import soundfile as sf


#### Audio Extraction from Video.

In [ ]:
def video_to_audio():

  folder_path = 'C:/Users/Harsh Patel/Desktop/Vintel/videos'
  for filename in os.listdir(folder_path):
      file_path = os.path.join(folder_path, filename)
      if os.path.isfile(file_path) and filename.endswith(('.mp4', '.mov', '.avi', '.mkv')):
            print(f"Processing video file: {filename}")

            # Open the video file and extract audio
            videoclip = VideoFileClip(file_path)
            audioclip = videoclip.audio

            # Write the audio file to mp3 format
            audio_output_path = f"C:/Users/Harsh Patel/Desktop/Vintel/audios/audiofile_{filename.split('.')[0]}.mp3"
            audioclip.write_audiofile(audio_output_path, codec="libmp3lame")
            print(f"Audio saved as: {audio_output_path}")


In [ ]:
# def transcript():

#   aai.settings.api_key = "70383e84e33044fa909ef8ae67a3aef2"
#   transcriber = aai.Transcriber()

#   folder_path = 'C:/Users/Harsh Patel/Desktop/Vintel/audios'
#   for filename in os.listdir(folder_path):
#       file_path = os.path.join(folder_path, filename)
#       if os.path.isfile(file_path) and filename.endswith(('.mp3')):

#         transcript = transcriber.transcribe(file_path)

#     # print(f"Processing audiofile_Video{i}")
#         transcript= transcript.text
#         with open(f'C:/Users/Harsh Patel/Desktop/Vintel/Transcripts/transcript_{filename}.txt', 'w') as f:
#                 f.write(transcript)



### Reading Audio Signal, and making a HuggingFace Dataset

In [ ]:
import os
from datasets import Dataset, DatasetDict
import soundfile as sf

def load_and_preprocess_data(audio_dir, transcript_dir):
    data = {'label': [], 'audio': [], 'transcript':[]}

    # Process audio files
    for index, audio_file in enumerate(os.listdir(audio_dir)):
        if audio_file.endswith('.mp3'):  # Adjust for audio file types
            file_path = os.path.join(audio_dir, audio_file)

            # Assuming preprocess_audio returns the processed audio and sample rate (sr)
#             processed_audio, sr = audio_reader(file_path)
            processed_audio, sample_rate = librosa.load(file_path, sr=16000, mono=True)

            # Append processed audio to the data dictionary
            data['audio'].append(processed_audio)

    # Process transcript files
    for index, transcript_file in enumerate(os.listdir(transcript_dir)):
        if transcript_file.endswith('.txt'):  # Adjust for transcript file types
            file_path = os.path.join(transcript_dir, transcript_file)

            # Read the transcript content and append it
            with open(file_path, 'r') as content:
                data['transcript'].append(content.read())

    # Ensure the label count matches the number of audios/transcripts
    num_entries = min(len(data['audio']), len(data['transcript']))  # Match the number of entries
    data['audio'] = data['audio'][:num_entries]  # Trim excess audio if necessary
    data['transcript'] = data['transcript'][:num_entries]  # Trim excess transcript if necessary

    data['label'] = [1] * num_entries  # Create a list of labels (e.g., all set to '1')

    return data

# Load and preprocess the audio and transcript data
audio_data1 = load_and_preprocess_data('/content/sample_data/audios', '/content/sample_data/transcripts')

train_dict = {k: v[:8] for k, v in audio_data1.items()}
test_dict = {k: v[8:] for k, v in audio_data1.items()}


# Convert it into a Hugging Face dataset
dataset1 = DatasetDict({
    'train': Dataset.from_dict(train_dict),
    'test': Dataset.from_dict(test_dict)# Split later into train/test if needed
})

print(dataset1)


DatasetDict({
    train: Dataset({
        features: ['label', 'audio', 'transcript'],
        num_rows: 8
    })
    test: Dataset({
        features: ['label', 'audio', 'transcript'],
        num_rows: 3
    })
})


### Model Training

### Speech 2 Text Model 1

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, pipeline

# Load your fine-tuned model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# Create the ASR pipeline with chunking enabled, explicitly providing feature_extractor and tokenizer
asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model=model,
    feature_extractor=processor.feature_extractor,  # Use the feature extractor
    tokenizer=processor.tokenizer,  # Use the tokenizer
    chunk_length_s=3  # Set chunk length in seconds (e.g., 5 seconds)
)

# Load the audio file path
audio_path = "/content/sample_data/audios/audio_1.mp3"

# Perform inference with chunking and streaming
transcription = asr_pipeline(audio_path)

# Print the final transcription
print("Transcription:", transcription['text'])


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequen

Transcription: OVER CAT TEACHER IS GOING TO TEACH YOU COSAN SIMILERTY AND COSAN DISTANCE HOW IT IS USIN TIT A SIENCE WILL LOOK AT SONTYRION THEN WE'LL MOON TO PIES AND GOL LET SE YOUR DID AS IENT IS WORKING FOR SOME FINANCIAL COMPANY WHERE ON A GUGAL DRIVE YOU HAVE BUNGE OF FINANCIAL DOCUMENTS NOW YOU DON'T KNOW FOR WI WHAT IS THE COMPANY ASSOCIATED WITH EACH OF THESE DOCUMENTS BUT WHEN YOU OPENED THE DOCUMEN WEN YOU READ IT YOU CAN KIND OF FIGURE OUT THAT THIS IS PROBABLY ABOUT APPLE WHY BECAUSE IPHON IS MENTIONED SO MANY TIMES SO WHEN YOU ARE READING ABOUT APL FINANCIAL REPORT THEY MIGHT MENTION GELIXY AS WELL BECAUSE THAT' THEIR MAIN COMPETITOR BUT HERE THE RITIO OF IPHON TO GELEXES IPHON HAVE TRI IPHON IS MENTIONED THREE TIMES GILIXIS MENTIONED ONE TIME SO YOU KNOW AL YOU SEE IPHONOCURRENCE MORE MUCH MORE THAN GALAXEY IF YOUR SAMSON DOCUMENT OF COURSE SAMSONCOM NEW DOCUMENT WILL HAVE MORE MENTION OF GALEXY THAN IV ON NOW LET'S SE NEW DOCUMENT COMES IN AND YOU DON'T KNOW WHAT IS THE

#### Using the model as it is but this time chunking the audio signal.

In [ ]:
# Chunking and streaming audio:

import librosa
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

# Load pretrained model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# Load audio and define chunk size (1-second chunks)
audio_path = "/content/sample_data/audios/audio_1.mp3"
audio_input, sample_rate = librosa.load(audio_path, sr=16000)

# Define chunk size (1 second of audio in 16000 Hz is 16000 samples)
chunk_size = 320000  # 1 second

# Process audio in chunks
transcription = ""
for i in range(0, len(audio_input), chunk_size):
    audio_chunk = audio_input[i:i+chunk_size]

    # Preprocess the chunk
    inputs = processor(audio_chunk, sampling_rate=16000, return_tensors="pt", padding=True)

    # Predict logits
    with torch.no_grad():
        logits = model(inputs.input_values).logits

    # Get predicted ids
    predicted_ids = torch.argmax(logits, dim=-1)

    # Decode the predictions
    transcription_chunk = processor.decode(predicted_ids[0])

    # Append the chunk's transcription to the full transcription
    transcription += transcription_chunk

print("Transcription:", transcription)


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

Transcription: AVAR CAT TEACHER IS GOING TO TEACH YOU KOSINSIMILARTY AND COSAN DISTANCE HOW IT IS USING DITTA SCIENCE WILL LOOK AT SOMTYRIAN THEN WE'LL MOON TO PIT AND COR TE SEE YOUR DITTA IENT IS WORKING FOR SOME FINANCIAL COMPANY WHERE ON A GUGALL DRIVE YOU HAVE BUNCH OF FINANCIAL DOCUMENTS NOW YOU DON'T KNOW FOR WITH WHAT IS THE COMPOLY ASSOCIATED WITH EACH OF THESE DOCUMENTS BUT WHEN YOU OPEN THE DOCUMENT WHEN YOU READ IT YOU CAN KIND OF FIGURE OUT THAT THIS IS PROBABLY ABOUT APPLE WHY BECAUSE IPONIS MENSIONED SO MANY TIMES SO WHEN YOU ARE READING ABOUT APPLE FINANCIAL REPORT THEY MIGHT MENTION GALECKSY AS WELL BECAUSEHAT THER MAIN COMPETITOR BUT HERE THE RICIO OF IPHON TO GELEXES IPHON HAVE TIPHON IS MENTIONED THREE TIMES GELXIS MENTIONED ONE TIME SO YOU KNOW AL YOU SEE IPHONACURRONCS MORE MUCH MORE THAN GELEXY IF YOUR SAMSON DOCUMENT OF COURSE SAMSON COME NEW DOCUMENT WILL HAVE MORE MENTION OF GELEXYTHEN IPHONE NOW LET'S SEE NEW DOCUMENT COMES IN AND YOU DON'T KNOW WHAT IS THE C

#### In the below code, we are doing chunking, and multi processing with the raw model. We also divide the into batch and process them.

In [ ]:
import librosa
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from multiprocessing import Pool, cpu_count

# Load pretrained model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# Ensure the model runs on CPU
model.to("cpu")

# Load audio and define chunk size (1-second chunks)
audio_path = "/content/sample_data/audios/audio_1.mp3"
audio_input, sample_rate = librosa.load(audio_path, sr=16000)

# Define chunk size (1 second of audio in 16000 Hz is 16000 samples)
chunk_size = 440000  # 1 second
batch_size = 2  # Number of chunks to process in a batch

# Split the audio into chunks
chunks = [audio_input[i:i+chunk_size] for i in range(0, len(audio_input), chunk_size)]

# Function to process a batch of audio chunks
def process_batch(batch):
    inputs = processor(batch, sampling_rate=16000, return_tensors="pt", padding=True)

    with torch.no_grad():
        logits = model(inputs.input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)

    # Decode the batch of predicted ids
    transcriptions = processor.batch_decode(predicted_ids)

    return transcriptions

# Function to handle multiprocessing for batches
def parallel_processing(chunks, batch_size):
    # Break the chunks into batches
    batches = [chunks[i:i + batch_size] for i in range(0, len(chunks), batch_size)]

    with Pool(cpu_count()) as pool:
        results = pool.map(process_batch, batches)

    # Combine all the transcriptions
    return " ".join([transcription for batch in results for transcription in batch])

# Execute the parallel processing
transcription = parallel_processing(chunks, batch_size)
print("Transcription:", transcription)


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

Transcription: AVAR CAT TEACHER IS GOING TO TEACH YOU COSINSIMILARITY AND COSAN DISTANCE HOW IT IS USIN DETA SINSE WELL LOOK AT SOME THYRIAN THEN WE'LL MOON TO PIT AND COR THEY SAE YOUR LETTES IENT IS WORKING FOR SOME FINANCIAL COMPANY WHERE ON A GUGALL DRIVE YOU HAVE BUNCH OF FINANCIAL DOCUMENTS NOW YOU DON'T KNOW FOR WI WHAT IS THE COMPANY ASSOCIATED WITH EACH OF THESE DOCUMENTS BUT WHEN YOU OPEN THE DOCUMENT WHEN YOU REAL IT YOU CAN KIND OF FIGURE OUT THAT THIS IS PROBABLY ABOUT APPLE WHY BECAUSE IPHON IS MENTIONED SO MANY TIMES SO WHEN YOU'RE READING ABOUT APPLE FINANCIAL REPORT THEY MIGHT MENTION GELEXY AS WELL BECAUSE THAT'S THEIR MAIN COMPETITOR BUT HERE THE RISIO OF IPHON TO GELEXEES IPHON HAV TR IPHON IS MENTIONED THREE TIMES GELXIS MENTIONED ONE TIME SO YOU KNOW ALL YOU SEE IPHONA CURRUNCS MORE MUCH MORE THAN GALAXEY IF YOU ARE SAM AND DOCUMENT OF COURSE SAMSON COM NE DOCUMENT WILL HAVE MORE MENTION OF GALEXY THAN IPHON NOW LET'S SEE NEW DOCUMENT COMES IN AND YOU DON'T KNOW W

#### In this approach we are using HuggingFace pipeline api to have a more structured. And, we are doing multiprocessing.

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, pipeline
import multiprocessing as mp
import librosa

# Load your fine-tuned model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# Create the ASR pipeline, explicitly providing feature_extractor and tokenizer
asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model=model,
    feature_extractor=processor.feature_extractor,  # Use the feature extractor
    tokenizer=processor.tokenizer  # Use the tokenizer
)

# Function to process the entire audio file using the ASR pipeline
def process_audio(audio_input):
    return asr_pipeline(audio_input)['text']

# Multiprocessing handler
def multiprocess_asr(audio_path, num_workers=mp.cpu_count()):
    # Load the full audio file
    audio_input, sample_rate = librosa.load(audio_path, sr=16000)

    # Use multiprocessing to process the entire audio file in parallel
    with mp.Pool(processes=num_workers) as pool:
        transcriptions = pool.map(process_audio, [audio_input])

    # Combine all transcriptions (though it's just one in this case)
    final_transcription = " ".join(transcriptions)
    return final_transcription

# Load the audio file path
audio_path = "/content/sample_data/audios/audio_1.mp3"

# Run the multiprocessing ASR inference
transcription = multiprocess_asr(audio_path)

# Print the final transcription
print("Transcription:", transcription)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequen

#### Wave2Vec2 Model without Multi Processing, Quantization or Chunking.

In [ ]:
# from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
# import torch
# import soundfile as sf
# import librosa

# # Load pretrained model and processor
# model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
# processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

# # Load your audio file (must be 16kHz)
# audio_input, sample_rate = librosa.load("/content/sample_data/audios/audio_1.mp3", sr=16000)

# # Preprocess the audio and convert it to input format
# inputs = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt", padding=True)
# # device = torch.device("cpu")
# # model.to(device)

# # Quantize the model
# model_quantized = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)

# import torch
# import os

# def get_model_size(model, filename='model.pt'):
#     torch.save(model.state_dict(), filename)
#     return os.path.getsize(filename)   # Size in MB

# original_size = get_model_size(model, 'original_model.pt')
# quantized_size = get_model_size(model_quantized, 'quantized_model.pt')

# print(f"Original Model Size: {original_size:.2f} MB")
# print(f"Quantized Model Size: {quantized_size:.2f} MB")

# # Get the logits (predictions) from the quantized model
# with torch.no_grad():
#     logits = model_quantized(inputs.input_values).logits

# # Decode the predicted text
# predicted_ids = torch.argmax(logits, dim=-1)
# transcription = processor.decode(predicted_ids[0])

# print("Transcription:", transcription)




### Tensorflow Approach

In [ ]:
from transformers import Speech2TextProcessor, TFSpeech2TextForConditionalGeneration


processor_tf = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")
model_tf = TFSpeech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")

In [ ]:
max_length = 1024  # Define a max length for the input sequence

input_features = processor_tf(
    dataset1['train']["audio"],
    sampling_rate=16000,
    return_tensors="tf",
    padding=True,  # Apply padding
    max_length=max_length,  # Truncate to max_length
    truncation=True  # Ensure truncation is applied
)

# Check input shape again
print(input_features['input_features'].shape)


(8, 1024, 80)


In [ ]:
max_length = 1024  # Define a max length for the input sequence

input_labels = processor_tf.tokenizer(
    dataset1['train']["transcript"],
#     sampling_rate=16000,
    return_tensors="tf",
    padding=True,  # Apply padding
    max_length=max_length,  # Truncate to max_length
    truncation=True  # Ensure truncation is applied
)

# Check input shape again
print(input_labels['input_ids'].shape)


(8, 1024)


In [ ]:
input_labels

{'input_ids': <tf.Tensor: shape=(8, 1024), dtype=int32, numpy=
array([[ 263,   15, 3259, ...,    1,    1,    1],
       [3774,   52,   60, ..., 1173,   36,    2],
       [  12,   67,  451, ...,    3,   31,    2],
       ...,
       [  52,   60,  243, ..., 1671,   25,    2],
       [  52,   67,  104, ...,    9,  453,    2],
       [ 216, 1378, 2957, ...,    4, 4906,    2]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(8, 1024), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int32)>}

In [ ]:
padded_inputs= np.array(input_features['input_features'], dtype= np.float32)
padded_labels = np.array(input_labels['input_ids'], dtype= np.float32)


In [ ]:
# padded_inputs

In [ ]:
# import tensorflow as tf
# dataset = tf.data.Dataset.from_tensor_slices(encoded_audio)

In [ ]:
print(padded_inputs.shape)  # Should output (num_samples, sequence_length, num_features)
print(padded_labels.shape)


(8, 1024, 80)
(8, 1024)


In [ ]:
import jiwer

def wer_metric(y_true, y_pred):
    # Convert the tensors to text sequences
    y_true_text = processor_tf.batch_decode(y_true, skip_special_tokens=True)
    y_pred_text = processor_tf.batch_decode(y_pred, skip_special_tokens=True)

    # Calculate WER using jiwer
    wer_score = jiwer.wer(y_true_text, y_pred_text)

    return wer_score


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Split dataset into training and testing
xtrain, xtest, ytrain, ytest = train_test_split(padded_inputs, padded_labels, test_size=0.2, shuffle=False)

# Define the model
model_asr = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(None, 80)),  # Input shape: (sequence_length, features)

    # Apply 1D convolutions over the input sequence
    tf.keras.layers.Conv1D(32, 3, activation='relu'),
    tf.keras.layers.Conv1D(64, 3, activation='relu'),

    # MaxPooling layer
    tf.keras.layers.MaxPooling1D(),

    # Dropout to prevent overfitting
    tf.keras.layers.Dropout(0.25),

    # Use GlobalAveragePooling1D to remove the need for a fixed input length
    tf.keras.layers.GlobalAveragePooling1D(),

    # Fully connected layers
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),

    # Output layer (adjust number of units according to your task)
    tf.keras.layers.Dense(8, activation='softmax')
])

# Print model summary
model_asr.summary()

# Define an optimizer and compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

# Compile the model



Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, None, 32)          7712      
                                                                 
 conv1d_9 (Conv1D)           (None, None, 64)          6208      
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, None, 64)          0         
 g1D)                                                            
                                                                 
 dropout_70 (Dropout)        (None, None, 64)          0         
                                                                 
 global_average_pooling1d_4  (None, 64)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_8 (Dense)             (None, 128)              

In [ ]:
ytrain[0].shape

(1024,)

In [ ]:
import keras

labels= [1,2,3,1,2,3]

# Define the custom loss function
def ctc_loss(y_true, y_pred):
    # Compute label lengths (for CTC)
    label_length = tf.math.count_nonzero(y_true, axis=1)  # Assuming padding for labels
    logit_length = tf.fill([tf.shape(y_pred)[0]], tf.shape(y_pred)[1])  # Logit length based on model output
    # Compute the CTC loss
    loss = tf.nn.ctc_loss(
        labels=y_true,
        logits=y_pred,
        label_length=label_length,
        logit_length=logit_length,
        blank_index=-1,  # Assuming the last index is reserved for the blank token in CTC
        logits_time_major=False  # Set to False if time dimension is not the first
    )
    return tf.reduce_mean(loss)

model_asr.compile(optimizer=optimizer, loss= tf.keras.losses.CategoricalCrossentropy())


# Train the model
model_asr.fit(xtrain, ytrain, epochs=3)

Epoch 1/3


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/usr/local/lib/python3.8/dist-packages/traitlets/config/application.py", line 1075, in launch_instance
      app.start()
    File "/usr/local/lib/python3.8/dist-packages/ipykernel/kernelapp.py", line 739, in start
      self.io_loop.start()
    File "/usr/local/lib/python3.8/dist-packages/tornado/platform/asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in r

In [ ]:
import tensorflow as tf
import tensorflow_io as tfio
import numpy as np

# Load dataset (for example, SpeechCommands)
# data_dir = "path_to_your_audio_dataset"
# train_ds = dataset1['train']['audio']

# Parameters
batch_size = 32
num_mfcc = 13
num_classes = 29  # Characters in alphabet + space + blank token

# Audio preprocessing function (e.g., converting to MFCC features)
def preprocess_audio(audio, sample_rate=16000):
    audio = tf.squeeze(audio, axis=-1)  # Remove last axis
    audio = tf.cast(audio, tf.float32)  # Normalize

    # Compute Short-time Fourier Transform (STFT)
    stft = tf.signal.stft(audio, frame_length=400, frame_step=160, fft_length=512)
    spectrogram = tf.abs(stft)

    # Compute Mel-scaled spectrogram
    num_mel_bins = 40
    lower_edge_hertz, upper_edge_hertz = 80.0, 4000.0
    linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
        num_mel_bins, spectrogram.shape[-1], sample_rate, lower_edge_hertz, upper_edge_hertz)
    mel_spectrogram = tf.tensordot(spectrogram, linear_to_mel_weight_matrix, 1)
    mel_spectrogram.set_shape(spectrogram.shape[:-1].concatenate([num_mel_bins]))

    # Compute log-mel spectrogram
    log_mel_spectrogram = tf.math.log(mel_spectrogram + 1e-6)

    # Compute MFCCs from log-mel spectrogram
    mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)[..., :num_mfcc]

    return mfccs

# Function to create dataset from audio and transcription files
def create_dataset(audio_files, transcripts, batch_size=32):
    def load_data(audio_file, transcript):
        audio = tf.io.read_file(audio_file)
        audio, _ = tf.audio.decode_wav(audio, desired_channels=1, desired_samples=16000)
        audio = preprocess_audio(audio)
        return audio, transcript

    dataset = tf.data.Dataset.from_tensor_slices((audio_files, transcripts))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.padded_batch(batch_size)
    return dataset

# Create your dataset here (audio_files and transcripts need to be preloaded)
audio_files = ['audio/audio_1.mp3', 'audio/audio_2.mp3', './audio_3.mp3', './audio_4.mp3', './audio_5.mp3', './audio_6.mp3', './audio_7.mp3', './audio_8.mp3', './audio_9.mp3', './audio_10.mp3', './audio_11.mp3']  # List of audio file paths
transcripts = ['transcripts/transcript1.txt', 'transcripts/transcript2.txt', 'transcripts/transcript3.txt', 'transcripts/transcript4.txt','transcripts/transcript5.txt', 'transcripts/transcript6.txt', 'transcripts/transcript7.txt', 'transcripts/transcript8.txt', 'transcripts/transcript9.txt', 'transcripts/transcript10.txt', 'transcripts/transcript11.txt']  # List of corresponding transcriptions
train_dataset = create_dataset(audio_files, transcripts)

# Build a simple model with CTC loss
def build_asr_model(input_dim, output_dim):
    inputs = tf.keras.Input(shape=(None, input_dim))  # Time dimension is variable
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(inputs)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(x)
    x = tf.keras.layers.Dense(output_dim + 1, activation='softmax')(x)  # +1 for blank token
    model = tf.keras.Model(inputs, x)
    return model

def ctc_loss_fn(y_true, y_pred):
    print("Shape of y_pred (logits):", y_pred.shape)  # Model output shape
    print("Shape of y_true (labels):", y_true.shape)  # Ground truth label shape

    input_length = tf.fill([tf.shape(y_pred)[0]], tf.shape(y_pred)[1])
    label_length = tf.reduce_sum(tf.cast(tf.not_equal(y_true, 0), tf.int32), axis=-1)

    print("Input length:", input_length)  # Should be the same length as y_pred's time dimension
    print("Label length:", label_length)  # Should be the actual lengths of the labels (excluding padding)

    loss = tf.nn.ctc_loss(
        labels=y_true,
        logits=y_pred,
        label_length=label_length,
        logit_length=input_length,
        logits_time_major=False,
        blank_index=-1
    )
    return tf.reduce_mean(loss)

# Compile the model
input_dim = num_mfcc
output_dim = num_classes
model = build_asr_model(input_dim, output_dim)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=ctc_loss_fn)

# Train the model
model.fit(train_dataset, epochs=10)

# For demonstration purposes, placeholders are used for `train_dataset`.


Epoch 1/10
Shape of y_pred (logits): (None, 98, 30)
Shape of y_true (labels): (None,)


TypeError: in user code:

    File "/usr/local/lib/python3.8/dist-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/tmp/ipykernel_858/334786748.py", line 71, in ctc_loss_fn  *
        label_length = tf.reduce_sum(tf.cast(tf.not_equal(y_true, 0), tf.int32), axis=-1)

    TypeError: Expected string passed to parameter 'y' of op 'NotEqual', got 0 of type 'int' instead. Error: Expected string, but got 0 of type 'int'.


### Speech 2 Text Model 2

In [ ]:
import torch
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration
from datasets import load_dataset

model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")

model_quantized = torch.quantization.quantize_dynamic(
    model,  # Model to quantize
    {torch.nn.Linear},  # Layers to quantize (Linear layers in this case)
    dtype=torch.qint8,
    inplace= True
)

# Ensure padding is enabled to avoid shape issues
inputs = processor(
    dataset1['train']["audio"],
    sampling_rate=16000,
    return_tensors="pt",
    padding=True  # Add padding so that all input sequences have the same length
)

with torch.no_grad():  # Disable gradient calculation for faster inference
    generated_ids = model.generate(
        inputs["input_features"],
        attention_mask=inputs["attention_mask"]
)

# # Now pass the input features and attention mask to the model for generation
# generated_ids = model.generate(
#     inputs["input_features"],
#     attention_mask=inputs["attention_mask"]

# Decode the generated ids to transcriptions
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)
transcription


Some weights of Speech2TextForConditionalGeneration were not initialized from the model checkpoint at facebook/s2t-small-librispeech-asr and are newly initialized: ['model.decoder.embed_positions.weights', 'model.encoder.embed_positions.weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

### Classification Model

In [ ]:
from transformers import Wav2Vec2Processor

# Load pre-trained processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

# Preprocess dataset to get audio input features
def preprocess_for_model(examples):
    audio = examples["audio"]
    inputs = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt")
    inputs["labels"] = torch.tensor(examples["label"], dtype=torch.long)  # Ensure labels are Long
    return inputs

# Apply the preprocessing to your dataset
encoded_dataset = dataset.map(preprocess_function, remove_columns=["audio"], batched=True)



In [ ]:
from transformers import Wav2Vec2ForSequenceClassification

# Load pre-trained Wav2Vec2 model for audio classification
model = Wav2Vec2ForSequenceClassification.from_pretrained(
    "facebook/wav2vec2-base-960h",
    num_labels=1  # 2 classes: speech and music
)


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./wav2vec2_output",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Create the trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],

    tokenizer=processor.feature_extractor,  # Use the Wav2Vec2 processor for tokenizing
)


In [ ]:
trainer.train()

results = trainer.evaluate()
print(results)


In [ ]:
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor

# Load the model and processor
model = Wav2Vec2ForSequenceClassification.from_pretrained("./saved_model")
processor = Wav2Vec2Processor.from_pretrained("./saved_model")

# Preprocess new audio for inference
def predict(audio_path):
    audio, sr = librosa.load(audio_path, sr=16000)
    inputs = processor(audio, sampling_rate=sr, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class = torch.argmax(logits, dim=-1).item()
    return predicted_class

# Example usage
audio_path = "/path/to/your/new_audio.wav"
prediction = predict(audio_path)
print("Prediction:", "Speech" if prediction == 0 else "Music")
